In [1]:
!pip install transformers
!pip install emoji
!pip install vncorenlp
!pip install sentence_transformers

In [2]:
import torch
import emoji
import re
import gc
import os
import json
import csv
import logging as lg
import pandas as pd
import numpy as np
import torch.nn.functional as F
import math
import transformers

from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from torch import nn
from transformers import *
from vncorenlp import VnCoreNLP
from nltk.tokenize import TweetTokenizer
from pandas import DataFrame

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
!nvidia-smi

Mon Apr  4 15:45:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    27W /  70W |   1482MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load data

In [5]:
train_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/public_train.csv"
val_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/val.csv"
test_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/final_private_test.csv"

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [7]:
def seed_everything(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [8]:
def save_checkpoint(model, tokenizer, checkpoint_path, epoch='best'):
    torch.save(model.state_dict(), os.path.join(
        checkpoint_path, f'model_{epoch}.bin'))
    model.config.to_json_file(os.path.join(checkpoint_path, 'config.json'))
    tokenizer.save_vocabulary(checkpoint_path)

## Models

In [9]:
class ElectraReINTELClassification(ElectraPreTrainedModel):
    def __init__(self, config):
        super(ElectraReINTELClassification, self).__init__(config=config)
        self.electra = ElectraModel(config)
        self.num_labels = config.num_labels
        self.init_weights()
        self.ln = torch.nn.Linear(config.hidden_size * 4, self.num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                start_positions=None, end_positions=None):
        outputs = self.electra(input_ids, attention_mask=attention_mask, position_ids=position_ids, head_mask=head_mask)[1]
        cls_output = torch.cat((outputs[-1][:, 0, ...], outputs[-2][:, 0, ...], outputs[-3][:, 0, ...], outputs[-4][:, 0, ...]), -1)
        logits = self.ln(cls_output)
        return logits

In [10]:
class RobertaReINTELClassification(BertPreTrainedModel):
    def __init__(self, config):
        super(RobertaReINTELClassification, self).__init__(config)
        self.roberta = RobertaModel(config)
        self.num_labels = config.num_labels
        self.outputs = torch.nn.Linear(config.hidden_size * 4, self.num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                start_positions=None, end_positions=None):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        cls_output = torch.cat((outputs[2][-1][:, 0, ...], outputs[2][-2][:, 0, ...], outputs[2][-3][:, 0, ...], outputs[2][-4][:, 0, ...]), -1)
        logits = self.outputs(cls_output)
        return logits

In [11]:
class BertReINTELClassification(BertPreTrainedModel):
    def __init__(self, config):
        super(BertReINTELClassification, self).__init__(config=config)
        self.bert = BertModel(config)
        self.num_labels = config.num_labels
        self.ln = torch.nn.Linear(config.hidden_size * 4, self.num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                start_positions=None, end_positions=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, position_ids=position_ids, head_mask=head_mask)[2]

        cls_output = torch.cat((outputs[-1][:, 0, ...], outputs[-2][:, 0, ...], outputs[-3][:, 0, ...], outputs[-4][:, 0, ...]), -1)
        logits = self.ln(cls_output)
        return logits

In [12]:
def normalizeToken(token):
    if len(token) == 1:
        return emoji.demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

## Preprocessing

In [13]:
def isnan(s):
    return s != s

# Tiền xử lý text
def normalizePost(post, tweet_tokenizer, vncorenlp, use_segment=False, remove_punc_stopword=False, lowercase_opt=False, truncation_method="head_only", length=256):
    post = post.strip()
    URL_pattern = r"(?:http?s://|www.)[^\"]+"
    hashtag_pattern = r"#\w+"

    post = re.sub(URL_pattern, "", post)
    post = re.sub(hashtag_pattern, "hashtag", post)
    post = re.sub("\.+",".", post)
    # post = re.sub("#\s+", " ", post)
    post = re.sub("\*+", " ", post)
    post = re.sub("\$+", "đô ", post)
    post = re.sub("-{2,}", "", post)
    post = re.sub("\@+", "", post)
    post = re.sub("\[[0-9]?[0-9]]", " : dẫn_chứng ", post)

    post = post.strip()
    if lowercase_opt:
      post = post.lower()
    tokens = tweet_tokenizer.tokenize(post.replace("’", "'").replace("…", "..."))
    
    post = " ".join(tokens)
    if use_segment:
        tokens = vncorenlp.tokenize(post.replace("’", "'").replace("…", "..."))
        tokens = [t for ts in tokens for t in ts]
    normPost = " ".join(tokens)

    if remove_punc_stopword:
      tokens = [t for t in normPost if not t in vnmese_stopwords]
    normPost = " ".join(tokens)

    normPost = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normPost)
    normPost = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normPost)
    normPost = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normPost)
    if use_segment:
        normPost = normPost.replace('< url >', '<url>')
        normPost = re.sub(r"# (\w+)", r'#\1', normPost)
    if truncation_method == "head_only":
      normPost = " ".join(normPost.split(" ")[:length])
    if truncation_method == "tail_only":
      normPost = " ".join(normPost.split(" ")[-length:])
    if truncation_method == "head_tail":
      normPost = " ".join(normPost.split(" ")[:int(length*0.25)]) + " " +  " ".join(normPost.split(" ")[-int(length*0.75):])

    replace_list = json.load(open("/content/drive/MyDrive/VLSP-Fake-News-Detection/replace_list.txt"))
    for k, v in replace_list.items():
        normPost = normPost.replace(k, v)
    return normPost

In [14]:
data_train = pd.read_csv(train_path)
data_val = pd.read_csv(val_path)
data_test = pd.read_csv(test_path)

In [15]:
data_train.dtypes

id                   int64
user_name           object
post_message        object
timestamp_post      object
num_like_post       object
num_comment_post    object
num_share_post      object
label                int64
dtype: object

In [16]:
#Time stamp feature
data_train['timestamp_post'] = pd.to_numeric(data_train['timestamp_post'], errors='coerce')
data_train['timestamp_post'] = data_train['timestamp_post'].astype('float')

data_test['timestamp_post'] = pd.to_numeric(data_test['timestamp_post'], errors='coerce')
data_test['timestamp_post'] = data_test['timestamp_post'].astype('float')

data_val['timestamp_post'] = pd.to_numeric(data_val['timestamp_post'], errors='coerce')
data_val['timestamp_post'] = data_val['timestamp_post'].astype('float')

#Number of shares feature
data_train['num_share_post'] = pd.to_numeric(data_train['num_share_post'], errors='coerce')
data_train['num_share_post'] = data_train['num_share_post'].astype('float')

data_test['num_share_post'] = pd.to_numeric(data_test['num_share_post'], errors='coerce')
data_test['num_share_post'] = data_test['num_share_post'].astype('float')

data_val['num_share_post'] = pd.to_numeric(data_val['num_share_post'], errors='coerce')
data_val['num_share_post'] = data_val['num_share_post'].astype('float')

#Number of likes feature
data_train['num_like_post'] = pd.to_numeric(data_train['num_like_post'], errors='coerce')
data_train['num_like_post'] = data_train['num_like_post'].astype('float')

data_test['num_like_post'] = pd.to_numeric(data_test['num_like_post'], errors='coerce')
data_test['num_like_post'] = data_test['num_like_post'].astype('float')

data_val['num_like_post'] = pd.to_numeric(data_val['num_like_post'], errors='coerce')
data_val['num_like_post'] = data_val['num_like_post'].astype('float')

#Number of comments feature
data_train['num_comment_post'] = pd.to_numeric(data_train['num_comment_post'], errors='coerce')
data_train['num_comment_post'] = data_train['num_comment_post'].astype('float')

data_test['num_comment_post'] = pd.to_numeric(data_test['num_comment_post'], errors='coerce')
data_test['num_comment_post'] = data_test['num_comment_post'].astype('float')

data_val['num_comment_post'] = pd.to_numeric(data_val['num_comment_post'], errors='coerce')
data_val['num_comment_post'] = data_val['num_comment_post'].astype('float')

data_train['post_message'] = data_train['post_message'].fillna('empty')
data_val['post_message'] = data_val['post_message'].fillna('empty')

In [17]:
data_train.isna().sum()

id                    0
user_name             0
post_message          0
timestamp_post      104
num_like_post       185
num_comment_post     83
num_share_post      800
label                 0
dtype: int64

In [18]:
def null_index_list(df, feature):
  return df[feature][df[feature].isnull()].index.tolist()

def fill_missing(df, feature):
  df[feature] = df[feature].fillna(df[feature].mean())

In [19]:
nan_timestamp_idxs = null_index_list(data_train, "timestamp_post")
nan_like_idxs = null_index_list(data_train, "num_like_post")
nan_share_idxs = null_index_list(data_train, "num_share_post")
nan_comment_idxs = null_index_list(data_train, "num_comment_post")

val_nan_timestamp_idxs = null_index_list(data_val, "timestamp_post")
val_nan_like_idxs = null_index_list(data_val, "num_like_post")
val_nan_share_idxs = null_index_list(data_val, "num_share_post")
val_nan_comment_idxs = null_index_list(data_val, "num_comment_post")

test_nan_timestamp_idxs = null_index_list(data_test, "timestamp_post")
test_nan_like_idxs = null_index_list(data_test, "num_like_post")
test_nan_share_idxs = null_index_list(data_test, "num_share_post")
test_nan_comment_idxs = null_index_list(data_test, "num_comment_post")

In [20]:
tweet_tokenizer = TweetTokenizer()
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/VLSP-Fake-News-Detection/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg")

train_sents = []
test_sents = []
val_sents = []

# If PhoBERT: use_segment = True, otherwise False
# normalizePost(post, tweet_tokenizer, vncorenlp, use_segment=False, remove_punc_stopword=False, lowercase_opt=False, truncation_method="head_only", length=512)
for post in data_train.post_message:
  train_sents.append(normalizePost(post, tweet_tokenizer, rdrsegmenter, False, False, False, "head_tail", 256))

for post in data_val.post_message:
  val_sents.append(normalizePost(post, tweet_tokenizer, rdrsegmenter, False, False, False, "head_tail", 256))

for test_post in data_test.post_message:
  test_sents.append(normalizePost(test_post, tweet_tokenizer, rdrsegmenter, False, False, False, "head_tail", 256))

In [21]:
from sentence_transformers import SentenceTransformer
sent_model = SentenceTransformer('bert-base-nli-mean-tokens')

train_sen_embeddings = sent_model.encode(train_sents)
test_sen_embeddings = sent_model.encode(test_sents)
val_sen_embeddings = sent_model.encode(val_sents)

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_bert-base-nli-mean-tokens/config.json
Model config BertConfig {
  "_name_or_path": "/root/.cache/torch/sentence_transformers/sentence-transformers_bert-base-nli-mean-tokens/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transformers_bert-base-nli-mean-tokens/pytorch_model.bin
All model checkpo

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
train_pairwise_similarities = cosine_similarity(train_sen_embeddings)
val_pairwise_similarities = cosine_similarity(val_sen_embeddings)
test_pairwise_similarities = cosine_similarity(test_sen_embeddings)

In [23]:
#Fill missing value by copy their most similar post message's values to them
def fill_missing_similarity(df, pairwise_similarity, feature, features_list):
  for i in features_list:
    max_val, pos_i, pos_j = -1, -1, -1
    for j in range(len(pairwise_similarity[i])):
      if i == j or j in features_list:
        continue
      elif pairwise_similarity[i][j] > max_val:
        #max_val = pairwise_similarities[i][j]
        pos_i = i
        pos_j = j
    df[feature][pos_i] = df[feature][pos_j]

In [24]:
fill_missing_similarity(data_train, train_pairwise_similarities, "timestamp_post", nan_timestamp_idxs)
fill_missing_similarity(data_val, val_pairwise_similarities, "timestamp_post", val_nan_timestamp_idxs)
fill_missing_similarity(data_test, test_pairwise_similarities, "timestamp_post", test_nan_timestamp_idxs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [25]:
fill_missing(data_train, "num_like_post")
fill_missing(data_train, "num_share_post")
fill_missing(data_train, "num_comment_post")

fill_missing(data_val, "num_like_post")
fill_missing(data_val, "num_share_post")
fill_missing(data_val, "num_comment_post")

fill_missing(data_test, "num_like_post")
fill_missing(data_test, "num_share_post")
fill_missing(data_test, "num_comment_post")

In [26]:
def create_additional_features(df):
  df['year'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.year
  df['month'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.month
  df['day'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.day
  df['weekday'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.weekday
  df['hour'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.hour

In [27]:
create_additional_features(data_train)
create_additional_features(data_val)
create_additional_features(data_test)

In [28]:
data_train = data_train.drop(columns=["timestamp_post"])
data_val = data_val.drop(columns=["timestamp_post"])
data_test = data_test.drop(columns=["timestamp_post"])

In [29]:
train_datetime_statements = []
for (day, month, hour, weekday) in zip(data_train['day'], data_train['month'], data_train['hour'], data_train['weekday']):
  if weekday != 6:
    train_pattern_post = f"Bài viết được đăng vào thứ {weekday + 2}, ngày {day} tháng {month} lúc {hour} giờ"
  else:
    train_pattern_post = f"Bài viết được đăng vào Chúa nhật, ngày {day} tháng {month} lúc {hour} giờ"
  train_datetime_statements.append(train_pattern_post)

print(train_datetime_statements)

['Bài viết được đăng vào thứ 6, ngày 3 tháng 4 lúc 20 giờ', 'Bài viết được đăng vào thứ 6, ngày 8 tháng 5 lúc 11 giờ', 'Bài viết được đăng vào thứ 7, ngày 6 tháng 6 lúc 1 giờ', 'Bài viết được đăng vào thứ 7, ngày 13 tháng 6 lúc 4 giờ', 'Bài viết được đăng vào thứ 2, ngày 9 tháng 3 lúc 7 giờ', 'Bài viết được đăng vào thứ 3, ngày 26 tháng 5 lúc 4 giờ', 'Bài viết được đăng vào thứ 4, ngày 13 tháng 5 lúc 19 giờ', 'Bài viết được đăng vào thứ 5, ngày 28 tháng 5 lúc 1 giờ', 'Bài viết được đăng vào thứ 4, ngày 15 tháng 4 lúc 11 giờ', 'Bài viết được đăng vào thứ 4, ngày 20 tháng 5 lúc 7 giờ', 'Bài viết được đăng vào thứ 5, ngày 21 tháng 5 lúc 5 giờ', 'Bài viết được đăng vào thứ 3, ngày 31 tháng 3 lúc 11 giờ', 'Bài viết được đăng vào thứ 2, ngày 13 tháng 4 lúc 1 giờ', 'Bài viết được đăng vào thứ 5, ngày 20 tháng 2 lúc 11 giờ', 'Bài viết được đăng vào thứ 6, ngày 26 tháng 6 lúc 8 giờ', 'Bài viết được đăng vào thứ 3, ngày 28 tháng 4 lúc 5 giờ', 'Bài viết được đăng vào thứ 4, ngày 29 tháng 4 lúc 0 

In [30]:
val_datetime_statements = []
for (day, month, hour, weekday) in zip(data_val['day'], data_val['month'], data_val['hour'], data_val['weekday']):
  if weekday != 6:
    val_pattern_post = f"Bài viết được đăng vào thứ {weekday + 2}, ngày {day} tháng {month} lúc {hour} giờ"
  else:
    val_pattern_post = f"Bài viết được đăng vào Chúa nhật, ngày {day} tháng {month} lúc {hour} giờ"
  val_datetime_statements.append(val_pattern_post)

In [31]:
test_datetime_statements = []
for (day, month, hour, weekday) in zip(data_test['day'], data_test['month'], data_test['hour'], data_test['weekday']):
  if weekday != 6:
    test_pattern_post = f"Bài viết được đăng vào thứ {weekday + 2}, ngày {day} tháng {month} lúc {hour} giờ"
  else:
    test_pattern_post = f"Bài viết được đăng vào Chúa nhật, ngày {day} tháng {month} lúc {hour} giờ"
  test_datetime_statements.append(test_pattern_post)

print(train_datetime_statements)

['Bài viết được đăng vào thứ 6, ngày 3 tháng 4 lúc 20 giờ', 'Bài viết được đăng vào thứ 6, ngày 8 tháng 5 lúc 11 giờ', 'Bài viết được đăng vào thứ 7, ngày 6 tháng 6 lúc 1 giờ', 'Bài viết được đăng vào thứ 7, ngày 13 tháng 6 lúc 4 giờ', 'Bài viết được đăng vào thứ 2, ngày 9 tháng 3 lúc 7 giờ', 'Bài viết được đăng vào thứ 3, ngày 26 tháng 5 lúc 4 giờ', 'Bài viết được đăng vào thứ 4, ngày 13 tháng 5 lúc 19 giờ', 'Bài viết được đăng vào thứ 5, ngày 28 tháng 5 lúc 1 giờ', 'Bài viết được đăng vào thứ 4, ngày 15 tháng 4 lúc 11 giờ', 'Bài viết được đăng vào thứ 4, ngày 20 tháng 5 lúc 7 giờ', 'Bài viết được đăng vào thứ 5, ngày 21 tháng 5 lúc 5 giờ', 'Bài viết được đăng vào thứ 3, ngày 31 tháng 3 lúc 11 giờ', 'Bài viết được đăng vào thứ 2, ngày 13 tháng 4 lúc 1 giờ', 'Bài viết được đăng vào thứ 5, ngày 20 tháng 2 lúc 11 giờ', 'Bài viết được đăng vào thứ 6, ngày 26 tháng 6 lúc 8 giờ', 'Bài viết được đăng vào thứ 3, ngày 28 tháng 4 lúc 5 giờ', 'Bài viết được đăng vào thứ 4, ngày 29 tháng 4 lúc 0 

In [32]:
train_num_statements = []
for (num_like, num_share, num_comment) in zip(data_train['num_like_post'], data_train['num_share_post'], data_train['num_comment_post']):
  # train_num_post = f"Bài viết có {num_like} lượt bình luận"
  train_num_post = f"Bài viết có {num_like} lượt thích, {num_share} lượt chia sẻ và {num_comment} bình luận"
  train_num_statements.append(train_num_post)

In [33]:
val_num_statements = []
for (num_like, num_share, num_comment) in zip(data_val['num_like_post'], data_val['num_share_post'], data_val['num_comment_post']):
  # val_num_post = f"Bài viết có {num_like} lượt bình luận"
  val_num_post = f"Bài viết có {num_like} lượt thích, {num_share} lượt chia sẻ và {num_comment} bình luận"
  val_num_statements.append(val_num_post)

In [34]:
test_num_statements = []
for (num_like, num_share, num_comment) in zip(data_test['num_like_post'], data_test['num_share_post'], data_test['num_comment_post']):
  # test_num_post = f"Bài viết có {num_like} lượt bình luận"
  test_num_post = f"Bài viết có {num_like} lượt thích, {num_share} lượt chia sẻ và {num_comment} bình luận"
  test_num_statements.append(test_num_post)

In [35]:
train_modified_texts = []
for (train_num_post, train_datetime_post, tr_sent) in zip(train_num_statements, train_datetime_statements, train_sents):
  # train_text = train_num_post + ". " + tr_sent
  train_text = train_num_post + ". " + train_datetime_post + ". " + tr_sent
  train_modified_texts.append(train_text)

In [36]:
val_modified_texts = []
for (val_num_post, val_datetime_post, val_sent) in zip(val_num_statements, val_datetime_statements, val_sents):
  val_text = val_num_post + ". " + val_datetime_post  + ". " + val_sent
  # val_text = val_num_post + ". " + val_sent
  val_modified_texts.append(val_text)

In [37]:
test_modified_texts = []
for (test_num_post, test_datetime_post, test_sent) in zip(test_num_statements, test_datetime_statements, test_sents):
  # test_text = test_num_post + ". " + test_sent
  test_text = test_num_post + ". " + test_datetime_post + ". " + test_sent
  test_modified_texts.append(test_text)

In [38]:
train_modified_texts[0]

'Bài viết có 19477.0 lượt thích, 173.0 lượt chia sẻ và 378.0 bình luận. Bài viết được đăng vào thứ 6, ngày 3 tháng 4 lúc 20 giờ. THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ , CHIẾN SỸ HY SINH Ở ĐÀ NẴNG Ngày 3/4 , Đại tướng Tô Lâm , Bộ trưởng Bộ Công an đã ký quyết định số 2398 / QĐ-BCA-X 01 thăng cấp bậc hàm từ Đại úy lên Thiếu tá đối với đồng chí Đặng Thanh Tuấn . Cùng ngày , Thiếu tướng Vũ Xuân , Giám đốc Công an thành phố Đà Nẵng ký Quyết định số 479 / QĐ-CATP thăng cấp bậc hàm từ Trung sĩ lên thượng sỹ đối với đồng chí Võ Văn Toàn . Đây là 2 cán bộ , chiến sỹ đã hy sinh trong quá trình thực hiện nhiệm vụ đảm bảo an ninh , trật tự , phòng , chống tội phạm trên địa bàn thành phố Đà Nẵng . Trước đó , vào lúc 20h40 ngày 2/4/2020 , Công an TP . Đà Nẵng nhận được tin báo của nhân dân có nhóm đối tượng đua xe và cướp giật người đi đường tại khu vực quận Sơn Trà , trong bối cảnh toàn quốc thực hiện cách ly xã hội theo Chỉ thị số 16 của Thủ tướng Chính phủ . Công an TP . Đà Nẵng đã chỉ đạo cho Công 

## Utils

In [39]:
def convert_tokens_to_ids(texts, tokenizer, max_seq_length=256, labels=None):
    input_ids, attention_masks = [], []
    for text in texts:
        inputs = tokenizer.encode_plus(text, padding='max_length', max_length=max_seq_length, truncation=True)
        input_ids.append(inputs['input_ids'])
        attention_masks.append(inputs['attention_mask'])

    if labels is not None:
        return torch.tensor(input_ids, dtype=torch.long), torch.tensor(attention_masks, dtype=torch.long), torch.tensor(
            labels, dtype=torch.long)
    return torch.tensor(input_ids, dtype=torch.long), torch.tensor(attention_masks, dtype=torch.long)

In [40]:
def eval(val_loader, model, epoch, device):
    # Evaluate model
    model.eval()
    y_val = []
    val_preds = None
    print(f"EPOCH {epoch + 1}: ===EVALUATION===")
    for (input_ids, attention_mask, y_batch) in val_loader:
        y_pred = model(input_ids.to(device),
                       attention_mask=attention_mask.to(device))
        y_pred = y_pred.squeeze().detach().cpu().numpy()
        val_preds = np.atleast_1d(y_pred) if val_preds is None else np.concatenate(
            [val_preds, np.atleast_1d(y_pred)])
        y_val.extend(y_batch.tolist())

    val_preds = sigmoid(val_preds)
    score = f1_score(y_val, val_preds > 0.5, pos_label=0)
    roc_score = roc_auc_score(y_val, val_preds)
    print(f"PREDICT {sum(val_preds <= 0.5)} INFORMATIVES")
    print(f"ACTUALY {len(y_val) - sum(y_val)} INFORMATIVES")

    print(
        f"\n----- F1 score @0.5 = {score:.4f}\nROC-AUC Score = {roc_score:.4f}")
    return roc_score

In [41]:
def predict(test_df, model, config, tweet_tokenizer, vncorenlp, model_tokenizer, test_modified_texts):
    test_post_ids = test_df.id.to_list()

    test_ids, test_masks = convert_tokens_to_ids(test_modified_texts, model_tokenizer, 256)

    test_dataset = TensorDataset(test_ids, test_masks)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model.eval()
    test_preds = None
    for i, (input_ids, masks) in enumerate(test_dataloader):
        if i % 20 == 0 or i == len(test_dataloader):
            print(f"Predicted {i} posts.")
        y_pred = model(input_ids.cuda(), attention_mask=masks.cuda())
        y_pred = y_pred.squeeze().detach().cpu().numpy()
        test_preds = np.atleast_1d(y_pred) if test_preds is None else np.concatenate([test_preds, np.atleast_1d(y_pred)])

    test_preds = sigmoid(test_preds)
    test_preds = test_preds.tolist()
    final_result = []
    for post_id, test_pred in zip(test_post_ids, test_preds):
      final_result.append([post_id, test_pred])
    
    result_df = DataFrame(final_result)
    result_df.to_csv('/content/drive/MyDrive/VLSP-Fake-News-Detection/results.csv', index=False)

## Hyper parameters

In [42]:
EPOCHS = 4
BATCH_SIZE = 32
ACCUMULATION_STEPS = 5
LEARNING_RATE = 1e-5

config_path = '/content/drive/MyDrive/VLSP-Fake-News-Detection/config/electra_1.json'
single_model_config = json.load(open(config_path, 'r'))

In [43]:
single_model_config

{'max_seq_length': 512,
 'model_name': 'FPTAI/velectra-base-discriminator-cased',
 'model_type': 'ELECTRA',
 'output_model': 'electra_1',
 'remove_punc_stopword': False,
 'use_wordsegment': False}

In [44]:
seed = 7370
seed_everything(seed)

## Training

In [45]:
if single_model_config['model_type'] == 'BERT':
  print("===Use BERT model===")
  checkpoint_dir = '/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/vbert_base/'
  tokenizer = BertTokenizer.from_pretrained(
      single_model_config['model_name'], do_lower_case=False)
  tokenizer.add_tokens(['<url>'])
  config = BertConfig.from_pretrained(single_model_config['model_name'], num_labels=1,
                                      output_hidden_states=True)
  model = BertReINTELClassification.from_pretrained(
      single_model_config['model_name'], config=config)
  model.to(device)
  tsfm = model.bert
elif single_model_config['model_type'] == 'ROBERTA':
  print("===Use PhoBERT model===")
  checkpoint_dir = '/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/phobert_base/'
  tokenizer = AutoTokenizer.from_pretrained(
      single_model_config['model_name'])
  tokenizer.add_tokens(['<url>'])
  config = RobertaConfig.from_pretrained(single_model_config['model_name'], num_labels=1,
                                          output_hidden_states=True)
  model = RobertaReINTELClassification.from_pretrained(
      single_model_config['model_name'], config=config)
  # model.resize_token_embeddings(len(tokenizer))
  model.to(device)
  tsfm = model.roberta
elif single_model_config['model_type'] == 'ELECTRA':
  print("===Use ELECTRA model===")
  checkpoint_dir = '/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra/'
  tokenizer = ElectraTokenizer.from_pretrained(
      single_model_config['model_name'], do_lower_case=False)
  tokenizer.add_tokens(['<url>'])
  config = ElectraConfig.from_pretrained(single_model_config['model_name'], num_labels=1,
                                          output_hidden_states=True, output_attentions=False)
  model = ElectraReINTELClassification.from_pretrained(
      single_model_config['model_name'], config=config)
  model.resize_token_embeddings(len(tokenizer))
  model.to(device)
  tsfm = model.electra
elif single_model_config['model_type'] == 'BERT4NEWS':
  print("===Use BERT4news model===")
  checkpoint_dir = '/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/bert4news/'
  tokenizer = BertTokenizer.from_pretrained(
      single_model_config['model_name'], do_lower_case=False)
  tokenizer.add_tokens(['<url>'])
  config = BertConfig.from_pretrained(single_model_config['model_name'], num_labels=1,
                                          output_hidden_states=True)
  model = Bert4newsReINTELClassification.from_pretrained(
      single_model_config['model_name'], config=config)
  #model.resize_token_embeddings(len(tokenizer))
  model.to(device)
  tsfm = model.bert4news
else:
  print("Model type invalid!!!")

print(f"Seed number: {seed}")

===Use ELECTRA model===


https://huggingface.co/FPTAI/velectra-base-discriminator-cased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpy4rj88sq


Downloading:   0%|          | 0.00/196k [00:00<?, ?B/s]

storing https://huggingface.co/FPTAI/velectra-base-discriminator-cased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/6474b234c96fb0f5976e19831e74198c32b39404aaa965825ab101fdcdeb2559.d3f67d031fef7c57be8879f51e1c247ae4df9a5af493b54bc1168b1fff91ec86
creating metadata file for /root/.cache/huggingface/transformers/6474b234c96fb0f5976e19831e74198c32b39404aaa965825ab101fdcdeb2559.d3f67d031fef7c57be8879f51e1c247ae4df9a5af493b54bc1168b1fff91ec86
loading file https://huggingface.co/FPTAI/velectra-base-discriminator-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/6474b234c96fb0f5976e19831e74198c32b39404aaa965825ab101fdcdeb2559.d3f67d031fef7c57be8879f51e1c247ae4df9a5af493b54bc1168b1fff91ec86
loading file https://huggingface.co/FPTAI/velectra-base-discriminator-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/FPTAI/velectra-base-discriminator-cased/resolve/main/special_tokens_map.json from cache 

Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

storing https://huggingface.co/FPTAI/velectra-base-discriminator-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/c9b8c963d8a6ceb38e7ff912a5a84ac9566b1adde45046d47ce23b06b0983d85.1627b686043688e70add2f147c8519e937f6995610cf029e09d244661ca757d7
creating metadata file for /root/.cache/huggingface/transformers/c9b8c963d8a6ceb38e7ff912a5a84ac9566b1adde45046d47ce23b06b0983d85.1627b686043688e70add2f147c8519e937f6995610cf029e09d244661ca757d7
loading configuration file https://huggingface.co/FPTAI/velectra-base-discriminator-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c9b8c963d8a6ceb38e7ff912a5a84ac9566b1adde45046d47ce23b06b0983d85.1627b686043688e70add2f147c8519e937f6995610cf029e09d244661ca757d7
Model config ElectraConfig {
  "_name_or_path": "FPTAI/velectra-base-discriminator-cased",
  "_num_labels": 2,
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

storing https://huggingface.co/FPTAI/velectra-base-discriminator-cased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/5d5e1722d1e5571a4ecff8c7dcfaab1f6765f498a7900989277d29ec5f6a21ba.4dceee0acbb0fe1bfb45dbe34f2d17bed5197b33417aa586d9030df3374d5d0f
creating metadata file for /root/.cache/huggingface/transformers/5d5e1722d1e5571a4ecff8c7dcfaab1f6765f498a7900989277d29ec5f6a21ba.4dceee0acbb0fe1bfb45dbe34f2d17bed5197b33417aa586d9030df3374d5d0f
loading weights file https://huggingface.co/FPTAI/velectra-base-discriminator-cased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/5d5e1722d1e5571a4ecff8c7dcfaab1f6765f498a7900989277d29ec5f6a21ba.4dceee0acbb0fe1bfb45dbe34f2d17bed5197b33417aa586d9030df3374d5d0f
Some weights of the model checkpoint at FPTAI/velectra-base-discriminator-cased were not used when initializing ElectraReINTELClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', '

Seed number: 7370


In [46]:
tr_labels = data_train.label.to_list()
train_ids, train_masks, train_labels = convert_tokens_to_ids(train_modified_texts, tokenizer, 256, tr_labels)
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
train_sampler = SequentialSampler(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)

In [47]:
vl_labels = data_val.label.to_list()
val_ids, val_masks, val_labels = convert_tokens_to_ids(val_modified_texts, tokenizer, 256, vl_labels)
val_dataset = TensorDataset(val_ids, val_masks, val_labels)
val_sampler = SequentialSampler(val_dataset)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [48]:
save_checkpoint(model, tokenizer, checkpoint_dir, epoch=seed)

In [49]:
num_train_optimization_steps = int(EPOCHS * len(train_dataset) / BATCH_SIZE / ACCUMULATION_STEPS)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(
        np in n for np in no_decay)], 'weight_decay': 0.005},
    {'params': [p for n, p in param_optimizer if any(
        np in n for np in no_decay)], 'weight_decay': 0.005}
]

optimizer = transformers.AdamW(optimizer_grouped_parameters,lr=LEARNING_RATE, correct_bias=False)
# optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr = LEARNING_RATE)
scheduler0 = transformers.get_constant_schedule(optimizer)
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=50, num_training_steps=num_train_optimization_steps)


for child in tsfm.children():
  for param in child.parameters():
    param.require_grad = False

#Convert to iterator
frozen = True
best_score = 0.0
for epoch in range(EPOCHS):
  if epoch > 0 and frozen:
    for child in tsfm.children():
      for param in child.parameters():
        param.requires_grad = True

    frozen = False
    del scheduler0
    torch.cuda.empty_cache()
    gc.collect()
  optimizer.zero_grad()    
  print(f'------ Start training on Epoch: {epoch + 1}/{EPOCHS}')
  avg_loss = 0
  avg_accuracy = 0
  model.train()
  for i, (input_ids, attention_mask, y_batch) in enumerate(train_loader):
    if(i % 20 == 0 and not i == 0) or (i == len(train_loader)):
      print(f"Training batch {i} of {len(train_loader)}")
    optimizer.zero_grad()
    y_pred = model(input_ids.to(device), attention_mask=attention_mask.to(device))
    loss = torch.nn.functional.binary_cross_entropy_with_logits(y_pred.view(-1).to(device), y_batch.float().to(device))
    loss = loss.mean()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    loss.backward()
    optimizer.step()

    lossf = loss.item()
    avg_loss += loss.item() / len(train_loader)

  if not frozen:
    scheduler.step()
  else:
    scheduler0.step()
  optimizer.zero_grad()

  roc_score = eval(val_loader, model, epoch, "cuda")
  if roc_score >= best_score:
    best_score = roc_score
    save_checkpoint(model, tokenizer, checkpoint_dir, epoch=seed)
    print("Updated best score model!!! -------<{}>" % best_score)
  print("==================Done=============")

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


------ Start training on Epoch: 1/4
Training batch 20 of 137
Training batch 40 of 137
Training batch 60 of 137
Training batch 80 of 137
Training batch 100 of 137
Training batch 120 of 137
EPOCH 1: ===EVALUATION===
PREDICT 875 INFORMATIVES
ACTUALY 731 INFORMATIVES

----- F1 score @0.5 = 0.9103
ROC-AUC Score = 0.4874
Updated best score model!!! -------<{}>
==================Done=============
------ Start training on Epoch: 2/4
Training batch 20 of 137
Training batch 40 of 137
Training batch 60 of 137
Training batch 80 of 137
Training batch 100 of 137
Training batch 120 of 137
EPOCH 2: ===EVALUATION===
PREDICT 729 INFORMATIVES
ACTUALY 731 INFORMATIVES

----- F1 score @0.5 = 0.9795
ROC-AUC Score = 0.9819
Updated best score model!!! -------<{}>
==================Done=============
------ Start training on Epoch: 3/4
Training batch 20 of 137
Training batch 40 of 137
Training batch 60 of 137
Training batch 80 of 137
Training batch 100 of 137
Training batch 120 of 137
EPOCH 3: ===EVALUATION===


In [50]:
if torch.cuda.is_available():
  device = torch.device('cuda')
  print(torch.cuda.get_device_name())
else:
  device = torch.device('cpu')
test_config_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra/model_7370.bin"
test_config = ElectraConfig.from_pretrained("/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra", num_labels=1, output_hidden_states=True)
test_model = ElectraReINTELClassification.from_pretrained(test_config_path, config=test_config)
test_model.to(device)
test_tokenizer = ElectraTokenizer.from_pretrained("/content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra", do_lower_case=False)

loading configuration file /content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra/config.json
Model config ElectraConfig {
  "_name_or_path": "FPTAI/velectra-base-discriminator-cased",
  "_num_labels": 2,
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.17.0",
  "t

Tesla T4


All model checkpoint weights were used when initializing ElectraReINTELClassification.

All the weights of ElectraReINTELClassification were initialized from the model checkpoint at /content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra/model_7370.bin.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ElectraReINTELClassification for predictions without further training.
Didn't find file /content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra/added_tokens.json. We won't load it.
Didn't find file /content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra/special_tokens_map.json. We won't load it.
Didn't find file /content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra/tokenizer_config.json. We won't load it.
loading file /content/drive/MyDrive/VLSP-Fake-News-Detection/trained_models/electra/vocab.txt
loading file None
loading file None
loading file None
loading configuration file 

In [52]:
predict(data_test, test_model, test_config, tweet_tokenizer, rdrsegmenter, test_tokenizer, test_modified_texts)

Predicted 0 posts.
Predicted 20 posts.
Predicted 40 posts.
